<a href="https://colab.research.google.com/github/Carter-and/Test-Branch/blob/master/Carter_facebook_ad_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This project is looking at Facebook ad data to tease out insights regarding how to advertise more effectively

In [0]:
import pandas as pd

In [0]:
DATA_URL = 'http://128.138.93.164/aprd6342/data/TravelPonyFacebook.csv'
df = pd.read_csv(DATA_URL)

In [0]:
df.head()

,Start Date,End Date,Account,Campaign Name,Reach,Frequency,Impressions,Clicks,Unique Clicks,Amount Spent (USD),Page Likes,Page Engagement,Post Engagement,Post Likes,Post Comments,Post Shares,Photo Views,Website Clicks,Cost per Page Like (USD),Cost per Page Engagement (USD)
0,2013-12-02,2013-12-02,Bob Jackson,Cyber Weekend,278117,2.314303,643647,336,313,300.00,3,246,243,0,0,0,0,243,100.00,1.219512
1,2013-12-01,2013-12-01,Bob Jackson,Cyber Weekend,221255,2.079090,460009,376,348,300.00,6,215,209,0,0,0,0,209,50.00,1.395349
2,2013-11-30,2013-11-30,Bob Jackson,Cyber Weekend,267915,2.499069,669538,383,345,300.00,2,252,250,0,0,0,0,250,150.00,1.190476
3,2013-11-29,2013-11-29,Bob Jackson,Cyber Weekend,280332,2.365406,663099,301,275,300.00,3,257,254,0,0,0,0,254,100.00,1.167315
4,2013-11-22,2013-11-22,Bob Jackson,TravelPony: General Ads,14269,1.000420,14275,64,61,35.18,2,14,12,0,0,0,0,12,17.59,2.512857


Cost per impression

In [0]:
df['CPI'] = df['Amount Spent (USD)']/df['Impressions']

In [0]:
df.head()

,Start Date,End Date,Account,Campaign Name,Reach,Frequency,Impressions,Clicks,Unique Clicks,Amount Spent (USD),Page Likes,Page Engagement,Post Engagement,Post Likes,Post Comments,Post Shares,Photo Views,Website Clicks,Cost per Page Like (USD),Cost per Page Engagement (USD),CPI
0,2013-12-02,2013-12-02,Bob Jackson,Cyber Weekend,278117,2.314303,643647,336,313,300.00,3,246,243,0,0,0,0,243,100.00,1.219512,0.000466
1,2013-12-01,2013-12-01,Bob Jackson,Cyber Weekend,221255,2.079090,460009,376,348,300.00,6,215,209,0,0,0,0,209,50.00,1.395349,0.000652
2,2013-11-30,2013-11-30,Bob Jackson,Cyber Weekend,267915,2.499069,669538,383,345,300.00,2,252,250,0,0,0,0,250,150.00,1.190476,0.000448
3,2013-11-29,2013-11-29,Bob Jackson,Cyber Weekend,280332,2.365406,663099,301,275,300.00,3,257,254,0,0,0,0,254,100.00,1.167315,0.000452
4,2013-11-22,2013-11-22,Bob Jackson,TravelPony: General Ads,14269,1.000420,14275,64,61,35.18,2,14,12,0,0,0,0,12,17.59,2.512857,0.002464


Considering CPI, what day of the week works best(cheapest)?
What day is worst (most expensive)?

In [0]:
df.head()

,Start Date,End Date,Account,Campaign Name,Reach,Frequency,Impressions,Clicks,Unique Clicks,Amount Spent (USD),Page Likes,Page Engagement,Post Engagement,Post Likes,Post Comments,Post Shares,Photo Views,Website Clicks,Cost per Page Like (USD),Cost per Page Engagement (USD)
0,2013-12-02,2013-12-02,Bob Jackson,Cyber Weekend,278117,2.314303,643647,336,313,300.00,3,246,243,0,0,0,0,243,100.00,1.219512
1,2013-12-01,2013-12-01,Bob Jackson,Cyber Weekend,221255,2.079090,460009,376,348,300.00,6,215,209,0,0,0,0,209,50.00,1.395349
2,2013-11-30,2013-11-30,Bob Jackson,Cyber Weekend,267915,2.499069,669538,383,345,300.00,2,252,250,0,0,0,0,250,150.00,1.190476
3,2013-11-29,2013-11-29,Bob Jackson,Cyber Weekend,280332,2.365406,663099,301,275,300.00,3,257,254,0,0,0,0,254,100.00,1.167315
4,2013-11-22,2013-11-22,Bob Jackson,TravelPony: General Ads,14269,1.000420,14275,64,61,35.18,2,14,12,0,0,0,0,12,17.59,2.512857


In [0]:
my_df = df

In [0]:
#use pandas to get weekdays

In [0]:
my_df['Start Date'] = pd.to_datetime(my_df['Start Date'], format="%Y-%m-%d")

my_df['Day of week (int)'] = my_df['Start Date'].dt.weekday

my_df['Day of week (str)'] = my_df['Start Date'].dt.day_name()


In [0]:
my_df.head()

,Start Date,End Date,Account,Campaign Name,Reach,Frequency,Impressions,Clicks,Unique Clicks,Amount Spent (USD),Page Likes,Page Engagement,Post Engagement,Post Likes,Post Comments,Post Shares,Photo Views,Website Clicks,Cost per Page Like (USD),Cost per Page Engagement (USD),Day of week (int),Day of week (str),CPI
0,2013-12-02,2013-12-02,Bob Jackson,Cyber Weekend,278117,2.314303,643647,336,313,300.00,3,246,243,0,0,0,0,243,100.00,1.219512,0,Monday,0.000466
1,2013-12-01,2013-12-01,Bob Jackson,Cyber Weekend,221255,2.079090,460009,376,348,300.00,6,215,209,0,0,0,0,209,50.00,1.395349,6,Sunday,0.000652
2,2013-11-30,2013-11-30,Bob Jackson,Cyber Weekend,267915,2.499069,669538,383,345,300.00,2,252,250,0,0,0,0,250,150.00,1.190476,5,Saturday,0.000448
3,2013-11-29,2013-11-29,Bob Jackson,Cyber Weekend,280332,2.365406,663099,301,275,300.00,3,257,254,0,0,0,0,254,100.00,1.167315,4,Friday,0.000452
4,2013-11-22,2013-11-22,Bob Jackson,TravelPony: General Ads,14269,1.000420,14275,64,61,35.18,2,14,12,0,0,0,0,12,17.59,2.512857,4,Friday,0.002464


In [0]:
my_df['CPI'] = my_df['Amount Spent (USD)']/df['Impressions']

When Considering Cost Per Impression, what day of the week works best?

In [0]:
my_df.groupby('Day of week (str)')[['CPI']].mean()

,CPI
Day of week (str),
Friday,0.004097
Monday,0.002910
Saturday,0.002629
Sunday,0.003688
Thursday,0.003398
Tuesday,0.003040
Wednesday,0.002981


This shows us that Saturday is the "cheapest" and Friday is the most expensive if we are looking at avg CPI


Now we are computing the correlation between Amount Spent and following variables:
 - Reach
 - Frequency
 - Unique Clicks
 - Page Likes

In [0]:
import statsmodels.api as sm

In [0]:
target = my_df['Amount Spent (USD)']
dep = my_df[['Reach', 'Frequency', 'Unique Clicks', 'Page Likes']]
mod = sm.OLS(target, dep) 
ols = mod.fit()
print(ols.summary())

                                 OLS Regression Results                                
Dep. Variable:     Amount Spent (USD)   R-squared (uncentered):                   0.905
Model:                            OLS   Adj. R-squared (uncentered):              0.905
Method:                 Least Squares   F-statistic:                              8800.
Date:                Wed, 13 Nov 2019   Prob (F-statistic):                        0.00
Time:                        18:28:46   Log-Likelihood:                         -13145.
No. Observations:                3705   AIC:                                  2.630e+04
Df Residuals:                    3701   BIC:                                  2.632e+04
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                    coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------

This shows us that Page Likes is affected the most by how much is spent on ads 

What variable most strongly predicts unique clicks? What does that mean practically?

In [0]:
target = my_df['Unique Clicks']
dep = my_df[['Reach', 'Frequency']]
mod = sm.OLS(target, dep)
ols = mod.fit()
print(ols.summary())


                                 OLS Regression Results                                
Dep. Variable:          Unique Clicks   R-squared (uncentered):                   0.557
Model:                            OLS   Adj. R-squared (uncentered):              0.556
Method:                 Least Squares   F-statistic:                              2325.
Date:                Wed, 13 Nov 2019   Prob (F-statistic):                        0.00
Time:                        18:28:34   Log-Likelihood:                         -15973.
No. Observations:                3705   AIC:                                  3.195e+04
Df Residuals:                    3703   BIC:                                  3.196e+04
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

According to this model, the frequency most strongly predicts unique clicks. Essentially, this means that how often an individual sees an ad is more predictive of unique click rather than simply increasing the number of people who see the ad.

In [0]:
#FIN